### Data Mining in RecordOwl (Silver 1)

In [34]:
# Standard library
import os
import glob
import re
import time
import asyncio

# Third-party HTTP / async
import requests
import aiohttp
import nest_asyncio

# Data & analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Web scraping
import scrapy
from scrapy_playwright.page import PageMethod
from bs4 import BeautifulSoup

# Fuzzy matching
from fuzzywuzzy import fuzz, process

# Apify
from apify_client import ApifyClient


### Ingesting from previous layer

In [35]:
parquet_path = "./Staging/Bronze/bronze_data_1.parquet"
if os.path.exists(parquet_path):
    acra_data_filtered_by_industry = pd.read_parquet(parquet_path, engine="fastparquet")
    print(f"Loaded {len(acra_data_filtered_by_industry)} rows from {parquet_path}")
    print(acra_data_filtered_by_industry.shape)
else:
    raise FileNotFoundError(f"Parquet file not found at {parquet_path}")

Loaded 2768 rows from ./Staging/Bronze/bronze_data_1.parquet
(2768, 14)


### Mining RecordOwl

In [ ]:
# COST-OPTIMIZED SCRAPER - V10 FIXED (No waitForTimeout)
from apify_client import ApifyClient
import pandas as pd
import time
import json
from requests.exceptions import RequestException

client = ApifyClient("")

BATCH_SIZE = 50
MAX_CONCURRENCY = 3
MAX_RETRIES = 3

def create_pagefunction_v9_fixed() -> str:
    """V9: Fixed - removed waitForTimeout, using proper Puppeteer methods"""
    return """
async function pageFunction(context) {
    const { page, log, request } = context;
    const uen = request?.userData?.uen || '';

    if (!uen) return { status: 'error', uen: null, error: 'Missing UEN' };

    try {
        // Wait for search results with fallback
        try {
            await page.waitForSelector('.flex-1.min-w-0', { timeout: 20000 });
        } catch (e) {
            await page.waitForSelector('a[href*="/company/"]', { timeout: 10000 });
        }

        // Find and click link with EXACT UEN match
        const linkClickResult = await page.evaluate((targetUen) => {
            const uenUpper = targetUen.toUpperCase();
            const allLinks = document.querySelectorAll('a[href*="/company/"]');

            if (allLinks.length === 0) {
                return { clicked: false, reason: 'No company links found' };
            }

            // First pass: EXACT UEN match
            for (const link of allLinks) {
                let parent = link.parentElement;
                for (let i = 0; i < 5 && parent; i++) {
                    const parentText = (parent.innerText || parent.textContent || '').toUpperCase();
                    const uenPattern = new RegExp('\\\\b' + uenUpper.replace(/[.*+?^${}()|[\\\\]\\\\]/g, '\\\\$&') + '\\\\b');
                    
                    if (uenPattern.test(parentText)) {
                        link.click();
                        return { clicked: true, href: link.getAttribute('href'), matchType: 'exact' };
                    }
                    parent = parent.parentElement;
                }
            }

            // Fallback: Click first result
            allLinks[0].click();
            return { clicked: true, href: allLinks[0].getAttribute('href'), matchType: 'fallback' };
        }, uen);

        if (!linkClickResult.clicked) {
            return { status: 'not_found', uen, error: linkClickResult.reason || 'No search results' };
        }

        // Wait for navigation
        try {
            await page.waitForNavigation({ waitUntil: 'domcontentloaded', timeout: 25000 });
        } catch (navError) {
            const currentUrl = page.url();
            if (!currentUrl.includes('/company/')) {
                return { status: 'error', uen, error: 'Navigation failed' };
            }
        }

        // Wait for content with multiple fallback selectors
        try {
            await page.waitForSelector('dt', { timeout: 15000 });
        } catch (e1) {
            try {
                await page.waitForSelector('dd', { timeout: 10000 });
            } catch (e2) {
                try {
                    await page.waitForSelector('a[href^="mailto:"]', { timeout: 5000 });
                } catch (e3) {
                    // Continue anyway - some pages might not have these elements
                }
            }
        }

        // Verify UEN on the company page
        const uenVerification = await page.evaluate((targetUen) => {
            const uenUpper = targetUen.toUpperCase();
            const pageText = (document.body.innerText || document.body.textContent || '').toUpperCase();
            const uenPattern = new RegExp('\\\\b' + uenUpper.replace(/[.*+?^${}()|[\\\\]\\\\]/g, '\\\\$&') + '\\\\b');
            
            let uenFound = uenPattern.test(pageText);
            let uenInStructure = false;
            
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (dtText.includes('uen') || dtText.includes('registration') || dtText.includes('business registration')) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') {
                        const ddText = dd.textContent.toUpperCase().trim();
                        if (uenPattern.test(ddText)) {
                            uenInStructure = true;
                        }
                    }
                }
            });

            return {
                uenFoundInPage: uenFound || uenInStructure,
                pageUrl: window.location.href
            };
        }, uen);

        // If UEN not found, return mismatch
        if (!uenVerification.uenFoundInPage) {
            return {
                status: 'uen_mismatch',
                uen,
                url: uenVerification.pageUrl,
                error: 'UEN not found on company page'
            };
        }

        // Extract data
        const data = await page.evaluate(() => {
            const SOCIAL_MEDIA_DOMAINS = ['facebook.com','linkedin.com','instagram.com','tiktok.com','twitter.com','x.com','youtube.com','pinterest.com'];
            
            // EMAIL
            const emails = [];
            document.querySelectorAll('a[href^="mailto:"]').forEach(a => {
                const email = a.href.replace('mailto:', '').trim();
                if (email && email.includes('@') && !emails.includes(email)) {
                    emails.push(email);
                }
            });
            
            // PHONE
            const phones = [];
            
            function formatSingaporePhone(text) {
                const digitsOnly = text.replace(/\\D/g, '');
                if (digitsOnly.length === 8) return '+65' + digitsOnly;
                if (digitsOnly.length === 10 && digitsOnly.startsWith('65')) return '+' + digitsOnly;
                if (digitsOnly.length === 11 && digitsOnly.startsWith('65')) return '+65' + digitsOnly.slice(2);
                if (digitsOnly.length >= 10) {
                    for (let i = 0; i <= digitsOnly.length - 10; i++) {
                        if (digitsOnly.slice(i, i+2) === '65' && digitsOnly.length - i >= 10) {
                            return '+' + digitsOnly.slice(i, i+10);
                        }
                    }
                }
                return null;
            }
            
            // tel: links
            document.querySelectorAll('a[href^="tel:"]').forEach(a => {
                const formatted = formatSingaporePhone(a.href.replace('tel:', '').trim());
                if (formatted && !phones.includes(formatted)) phones.push(formatted);
            });
            
            // dt/dd structure
            const phoneKeywords = ['company contact', 'business contact', 'office phone', 'main phone', 'business phone', 'company phone', 'contact number', 'phone', 'tel', 'mobile', 'call', 'contact no'];
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (phoneKeywords.some(kw => dtText.includes(kw))) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') {
                        const formatted = formatSingaporePhone(dd.textContent.trim());
                        if (formatted && !phones.includes(formatted)) phones.push(formatted);
                    }
                }
            });
            
            // Pattern matching
            const bodyText = document.body.innerText || document.body.textContent;
            const phonePatterns = [
                /\\b(\\+65[\\s\\-]?)?([689]\\d{3}[\\s\\-]?\\d{4})\\b/g,
                /\\b65[\\s\\-]?([689]\\d{3})[\\s\\-]?(\\d{4})\\b/g,
                /\\b([689]\\d{3})[\\s\\-](\\d{4})\\b/g
            ];
            phonePatterns.forEach(pattern => {
                const matches = bodyText.matchAll(pattern);
                for (const match of matches) {
                    const formatted = formatSingaporePhone(match[0]);
                    if (formatted && !phones.includes(formatted)) phones.push(formatted);
                }
            });
            
            // WEBSITE
            const websites = [];
            document.querySelectorAll('a[href^="http"]').forEach(a => {
                const href = a.href.trim().toLowerCase();
                if (!SOCIAL_MEDIA_DOMAINS.some(d => href.includes(d)) && !href.includes('recordowl') && !href.includes('apify')) {
                    if (href.match(/\\.(com|sg|net|org|co)/)) websites.push(a.href);
                }
            });
            
            // SOCIAL MEDIA
            const facebook = [], linkedin = [], instagram = [], tiktok = [];
            document.querySelectorAll('a[href*="facebook.com"]').forEach(a => { if (!facebook.includes(a.href)) facebook.push(a.href); });
            document.querySelectorAll('a[href*="linkedin.com"]').forEach(a => { if (!linkedin.includes(a.href)) linkedin.push(a.href); });
            document.querySelectorAll('a[href*="instagram.com"]').forEach(a => { if (!instagram.includes(a.href)) instagram.push(a.href); });
            document.querySelectorAll('a[href*="tiktok.com"]').forEach(a => { if (!tiktok.includes(a.href)) tiktok.push(a.href); });
            
            // ADDRESS
            let address = null;
            const addressLabels = ['registered address', 'registered office address', 'address', 'principal place of business'];
            document.querySelectorAll('dt').forEach(dt => {
                const dtText = dt.textContent.toLowerCase().trim();
                if (addressLabels.some(label => dtText.includes(label))) {
                    const dd = dt.nextElementSibling;
                    if (dd && dd.tagName === 'DD') address = dd.textContent.trim();
                }
            });

            return {
                emails: emails.length ? emails : null,
                phones: phones.length ? phones : null,
                website: websites.length ? websites[0] : null,
                facebook: facebook.length ? facebook : null,
                linkedin: linkedin.length ? linkedin : null,
                instagram: instagram.length ? instagram : null,
                tiktok: tiktok.length ? tiktok : null,
                address: address
            };
        });

        return { status: 'success', uen, url: page.url(), ...data };

    } catch (err) {
        return { status: 'error', uen, error: err.message };
    }
}
"""

def run_scraper(client, uens):
    start_urls = [{"url": f"https://recordowl.com/search?name={uen}", "userData": {"uen": uen}} for uen in uens]

    run_input = {
    "startUrls": start_urls,
    "useChrome": False,
    "headless": True,
    "stealth": False,  # CHANGED: Removed stealth overhead
    "pageFunction": create_pagefunction_v9_fixed(),
    "maxRequestRetries": MAX_RETRIES,
    "maxRequestsPerCrawl": len(start_urls),
    "maxConcurrency": MAX_CONCURRENCY,
    "memoryMbytes": 2048,  # NEW: Reduced from 4096 MB
    "pageLoadTimeoutSecs": 25,  # CHANGED: Reduced from 40
    "pageFunctionTimeoutSecs": 60,  # CHANGED: Reduced from 120
    "waitUntil": ["domcontentloaded"],
    "proxyConfiguration": {"useApifyProxy": True},  # CHANGED: Datacenter instead of residential
}

    try:
        run = client.actor("apify/puppeteer-scraper").call(run_input=run_input)
        if not run or not isinstance(run, dict) or 'id' not in run:
            return [], "API returned invalid response"
    except Exception as e:
        return [], f"API call failed: {str(e)}"

    try:
        run_client = client.run(run["id"])
        run_info = run_client.wait_for_finish()
        status = run_info.get('status', 'UNKNOWN')

        if status in ['FAILED', 'TIMED-OUT', 'ABORTED']:
            return [], f"Actor run {status}"

        if status == "SUCCEEDED" and "defaultDatasetId" in run:
            dataset = client.dataset(run["defaultDatasetId"])
            items = list(dataset.iterate_items())
            return items, None

        return [], f"Scraping failed with status: {status}"
    except Exception as e:
        return [], f"Run monitoring error: {str(e)}"


# Execute scraper
all_results = []
total_rows = len(acra_data_filtered_by_industry)
total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE

print(f"Starting scraper - {total_rows} UENs in {total_batches} batches")

for batch_idx in range(0, total_rows, BATCH_SIZE):
    batch = acra_data_filtered_by_industry.iloc[batch_idx:batch_idx + BATCH_SIZE]
    uens = [str(row['UEN']).strip() for _, row in batch.iterrows()]
    
    batch_num = (batch_idx//BATCH_SIZE)+1
    print(f"\nBatch {batch_num}/{total_batches} - {len(uens)} UENs...")
    
    items, error = run_scraper(client, uens)
    
    if error:
        print(f"  Error: {error}")
        for uen in uens:
            all_results.append({
                "UEN": uen, "Status": "error", "Error": error,
                **{k: None for k in ['Emails','Phones','Website','Facebook','LinkedIn','Instagram','TikTok','address','RecordOwl_Link']}
            })
        continue
    
    uen_map = {item.get('uen'): item for item in items if item.get('uen')}
    
    success_count = 0
    for uen in uens:
        item = uen_map.get(uen)
        if not item:
            all_results.append({
                "UEN": uen, "Status": "missing", "Error": "No data returned",
                **{k: None for k in ['Emails','Phones','Website','Facebook','LinkedIn','Instagram','TikTok','address','RecordOwl_Link']}
            })
        else:
            status = item.get('status', 'error')
            if status == 'success':
                success_count += 1
            
            all_results.append({
                'UEN': uen,
                'Status': status,
                'Error': item.get('error'),
                'Emails': item.get('emails'),
                'Phones': item.get('phones'),
                'Website': item.get('website'),
                'Facebook': item.get('facebook'),
                'LinkedIn': item.get('linkedin'),
                'Instagram': item.get('instagram'),
                'TikTok': item.get('tiktok'),
                'address': item.get('address'),
                'RecordOwl_Link': item.get('url')
            })
    
    print(f"  Success: {success_count}/{len(uens)}")
    
    if batch_num < total_batches:
        time.sleep(3)

New_Fresh_Leads = pd.DataFrame(all_results)

Starting scraper - 2768 UENs in 56 batches

Batch 1/56 - 50 UENs...


[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> Status: RUNNING, Message: Starting the crawler.
[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> 2026-01-03T06:04:41.786Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> 2026-01-03T06:04:41.787Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> 2026-01-03T06:04:41.826Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> 2026-01-03T06:04:42.011Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:KPqxJu2pcXnCsFQzB] -> 2026-01-03T06:04:43.558Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppe

  Success: 50/50

Batch 2/56 - 50 UENs...


[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> 2026-01-03T06:09:36.802Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> 2026-01-03T06:09:36.804Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> 2026-01-03T06:09:36.844Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> 2026-01-03T06:09:37.095Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> 2026-01-03T06:09:37.731Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:ziktwOdqsm4ztGT8z] -> 2026-01-03T06:09:37.860Z INFO  Configuring Puppeteer Scraper

  Success: 49/50

Batch 3/56 - 50 UENs...


...

  Success: 50/50

Batch 55/56 - 50 UENs...


[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> 2026-01-03T10:18:58.564Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> 2026-01-03T10:18:58.565Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> 2026-01-03T10:18:58.981Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> 2026-01-03T10:18:59.300Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> 2026-01-03T10:19:00.213Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:N4U1E4Ld6mR6Kgd7A] -> 2026-01-03T10:19:00.359Z INFO  Configuring Puppeteer Scraper

  Success: 50/50

Batch 56/56 - 18 UENs...


[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> 2026-01-03T10:23:52.387Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> 2026-01-03T10:23:52.389Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> 2026-01-03T10:23:52.447Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> 2026-01-03T10:23:52.637Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> 2026-01-03T10:23:53.458Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:3eUaldyOu2NteKjiE] -> 2026-01-03T10:23:53.572Z INFO  Configuring Puppeteer Scraper

  Success: 18/18


In [37]:
print(f"\n{'='*70}")
print(f"✅ SCRAPING COMPLETE")
print(f"{'='*70}")
print(f"📊 Results Summary:")
print(f"   • Total processed: {len(New_Fresh_Leads)}")
print(f"   • Successful: {(New_Fresh_Leads['Status']=='success').sum()}")
print(f"   • Failed: {(New_Fresh_Leads['Status']=='error').sum()}")
print(f"   • Missing: {(New_Fresh_Leads['Status']=='missing').sum()}")
print(f"\n📞 Data Extracted:")
print(f"   • Phones: {New_Fresh_Leads['Phones'].notna().sum()}")
print(f"   • Emails: {New_Fresh_Leads['Emails'].notna().sum()}")
print(f"   • Websites: {New_Fresh_Leads['Website'].notna().sum()}")
print(f"   • Facebook: {New_Fresh_Leads['Facebook'].notna().sum()}")
print(f"   • Instagram: {New_Fresh_Leads['Instagram'].notna().sum()}")
print(f"   • LinkedIn: {New_Fresh_Leads['LinkedIn'].notna().sum()}")
print(f"   • TikTok: {New_Fresh_Leads['TikTok'].notna().sum()}")
print(f"{'='*70}")

New_Fresh_Leads.head(10)


✅ SCRAPING COMPLETE
📊 Results Summary:
   • Total processed: 2768
   • Successful: 2701
   • Failed: 50
   • Missing: 4

📞 Data Extracted:
   • Phones: 1137
   • Emails: 459
   • Websites: 1143
   • Facebook: 794
   • Instagram: 918
   • LinkedIn: 42
   • TikTok: 0


,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,address,RecordOwl_Link
0,202540984R,success,None,None,None,None,None,None,None,None,711 GEYLANG ROAD #01-02 ORIENTAL VENTURE BUILD...,https://recordowl.com/company/harmony-unisex-b...
1,53404596X,success,None,None,[+6597785712],None,[https://www.facebook.com/fancynailsparadise/],None,[https://www.instagram.com/fancynailsparadise/],None,756 UPPER SERANGOON ROAD #02-34 UPPER SERANGOO...,https://recordowl.com/company/fancy-nail-studio
2,201829681R,success,None,None,None,None,None,None,None,None,530 BEDOK NORTH STREET 3 #01-146 SINGAPORE 460530,https://recordowl.com/company/sensation-gaea-p...
3,201735516R,success,None,[soulartnailsandacademy@gmail.com],[+6596390264],None,[https://www.facebook.com/soulartnails/],None,"[https://instagram.com/p/B0fd1yEH5OC, https://...",None,100 ORCHARD ROAD #02-64 CONCORDE HOTEL AND SHO...,https://recordowl.com/company/soul-art-nails-p...
4,53404331C,success,None,None,[+6562351125],https://myqbeaute.com/,None,None,[https://www.instagram.com/qbeautywellness/?hl...,None,14 SCOTTS ROAD #05-120 FAR EAST PLAZA SINGAPOR...,https://recordowl.com/company/q-beauty
5,202515699G,success,None,None,[+6562220308],https://www.pureblissspa.com.sg/,None,None,None,None,177 TOA PAYOH CENTRAL\n#01-146\nSingapore 310177,https://recordowl.com/company/pure-bliss-beaut...
6,202443685H,success,None,None,None,https://www.mycareersfuture.gov.sg/job/persona...,None,None,None,None,133 NEW BRIDGE ROAD #10-03 CHINATOWN POINT SIN...,https://recordowl.com/company/esteem-beauty-we...
7,53482810L,success,None,None,None,http://www.deglow.co/,None,None,None,None,440A FERNVALE LINK #25-185 FERNVALE GARDENS 79...,https://recordowl.com/company/deglow
8,53505598J,success,None,None,None,https://www.mycareersfuture.gov.sg/job/persona...,None,None,None,None,52 PRINSEP STREET #01-01 PRINSEP PLACE SINGAPO...,https://recordowl.com/company/yi-xin-beaute
9,202505543E,success,None,None,None,https://www.mycareersfuture.gov.sg/job/general...,None,None,None,None,100 JALAN SULTAN\n#04-27\nSULTAN PLAZA\nSingap...,https://recordowl.com/company/bao-li-tang-02-p...


### Address Formatting 

In [38]:
# Pre-compile patterns for speed
POSTAL_RE = re.compile(r"(?:\bSingapore\b\s*)?(?P<postal>\d{6})(?!\d)", re.IGNORECASE)
UNIT_RES = [
    re.compile(r"#\s*[A-Za-z0-9]{1,4}\s*[-–]\s*[A-Za-z0-9]{1,4}", re.IGNORECASE),
    re.compile(r"\bunit\s*[#:]?\s*[A-Za-z0-9]{1,4}\s*[-–]\s*[A-Za-z0-9]{1,4}\b", re.IGNORECASE),
    re.compile(r"\bunit\s*[#:]?\s*[A-Za-z0-9]{1,5}\b", re.IGNORECASE),
]

def normalize_spaces(text: str) -> str:
    text = re.sub(r"[\n\r\t]+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip(" ,;|/")

def extract_postal(text: str) -> tuple[str, str | None]:
    if not text:
        return text, None
    matches = list(POSTAL_RE.finditer(text))
    if matches:
        m = matches[-1]
        postal = m.group("postal")
        start, end = m.span()
        cleaned = text[:start] + text[end:]
        cleaned = re.sub(r"\bSingapore\b", "", cleaned, flags=re.IGNORECASE)
        return normalize_spaces(cleaned), postal
    return normalize_spaces(text), None

def extract_unit(text: str) -> tuple[str, str | None]:
    if not text:
        return text, None
    for rx in UNIT_RES:
        m = rx.search(text)
        if m:
            unit_raw = m.group(0)
            cleaned = normalize_spaces(text[:m.start()] + text[m.end():])
            unit_digits = re.sub(r"^unit\s*[#:]?\s*", "", unit_raw, flags=re.IGNORECASE)
            unit_digits = normalize_spaces(unit_digits)
            unit_digits = unit_digits.replace(' – ', '-').replace('–', '-').replace(' ', '')
            unit_digits = unit_digits.lstrip('#')
            return cleaned, unit_digits
    return normalize_spaces(text), None

def clean_street(text: str) -> str | None:
    if not text:
        return None
    text = normalize_spaces(text)
    text = re.sub(r"\s*,\s*", ", ", text)
    return text if text.isupper() else text.title()

def split_address_sg(address: str) -> dict:
    if not isinstance(address, str) or not address.strip():
        return {"street": None, "unit": None, "postal_code": None, "address_clean": None}
    raw = normalize_spaces(address)
    without_postal, postal = extract_postal(raw)
    without_unit, unit = extract_unit(without_postal)
    without_unit = normalize_spaces(re.sub(r"\bSingapore\b", "", without_unit, flags=re.IGNORECASE))
    street = clean_street(without_unit)
    address_clean = normalize_spaces(" ".join(x for x in [street or "", unit or "", f"Singapore {postal}" if postal else ""] if x))
    return {"street": street, "unit": unit, "postal_code": postal, "address_clean": address_clean}

# Apply to current result DF -> create a new dataframe with clean components
if 'address' not in New_Fresh_Leads.columns:
    raise ValueError("Column 'address' not found in New_Fresh_Leads. Run the scraping cell first.")

parsed_df = pd.DataFrame(list(New_Fresh_Leads["address"].apply(split_address_sg)))

# New DataFrame with clean address fields and without raw 'address'
Cleaned_New_Fresh_Leads = New_Fresh_Leads.copy()
if 'address' in Cleaned_New_Fresh_Leads.columns:
    Cleaned_New_Fresh_Leads = Cleaned_New_Fresh_Leads.drop(columns=['address'])
Cleaned_New_Fresh_Leads["operational_street"] = parsed_df["street"]
Cleaned_New_Fresh_Leads["operational_unit"] = parsed_df["unit"]
Cleaned_New_Fresh_Leads["operational_postal_code"] = parsed_df["postal_code"]
Cleaned_New_Fresh_Leads["operational_address"] = parsed_df["address_clean"]

# Save full result to a new DataFrame and display all columns
New_Fresh_Leads_Operational = Cleaned_New_Fresh_Leads.copy()
New_Fresh_Leads_Operational

,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,operational_street,operational_unit,operational_postal_code,operational_address
0,202540984R,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/harmony-unisex-b...,711 GEYLANG ROAD ORIENTAL VENTURE BUILDING,01-02,389626,711 GEYLANG ROAD ORIENTAL VENTURE BUILDING 01-...
1,53404596X,success,None,None,[+6597785712],None,[https://www.facebook.com/fancynailsparadise/],None,[https://www.instagram.com/fancynailsparadise/],None,https://recordowl.com/company/fancy-nail-studio,756 UPPER SERANGOON ROAD UPPER SERANGOON SHOPP...,02-34,534626,756 UPPER SERANGOON ROAD UPPER SERANGOON SHOPP...
2,201829681R,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/sensation-gaea-p...,530 BEDOK NORTH STREET 3,01-146,460530,530 BEDOK NORTH STREET 3 01-146 Singapore 460530
3,201735516R,success,None,[soulartnailsandacademy@gmail.com],[+6596390264],None,[https://www.facebook.com/soulartnails/],None,"[https://instagram.com/p/B0fd1yEH5OC, https://...",None,https://recordowl.com/company/soul-art-nails-p...,100 ORCHARD ROAD CONCORDE HOTEL AND SHOPPING MALL,02-64,238840,100 ORCHARD ROAD CONCORDE HOTEL AND SHOPPING M...
4,53404331C,success,None,None,[+6562351125],https://myqbeaute.com/,None,None,[https://www.instagram.com/qbeautywellness/?hl...,None,https://recordowl.com/company/q-beauty,14 SCOTTS ROAD FAR EAST PLAZA,05-120,228213,14 SCOTTS ROAD FAR EAST PLAZA 05-120 Singapore...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,202223337Z,success,None,[mailbox@kallos.com.sg],None,https://www.kallosbeautyco.com/,None,[https://sg.linkedin.com/company/kallos-magazine],[https://www.instagram.com/kallosmag/?hl=en],None,https://recordowl.com/company/kallos-beauty-pt...,3 ANG MO KIO STREET 62 LINK@AMK,08-02,569139,3 ANG MO KIO STREET 62 LINK@AMK 08-02 Singapor...
2764,201604454R,success,None,None,[+6565625210],https://www.mycareersfuture.gov.sg/job/persona...,[https://www.facebook.com/evolution.hub.1/],None,None,None,https://recordowl.com/company/8-aesthetics-int...,211 UPPER BUKIT TIMAH ROAD #03- BANYAN TREE HOUSE,03-01,588182,211 UPPER BUKIT TIMAH ROAD #03- BANYAN TREE HO...
2765,202321966E,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/beauty-strokes-e...,66 TANNERY LANE SINDO INDUSTRIAL BUILDING,01-03L,347805,66 TANNERY LANE SINDO INDUSTRIAL BUILDING 01-0...
2766,202524406M,success,None,None,None,None,None,None,None,None,https://recordowl.com/company/ngcompany-pte-ltd,606D TAMPINES STREET 61 TAMPINES GREENRIDGES,14-364,524606,606D TAMPINES STREET 61 TAMPINES GREENRIDGES 1...


### Check for duplication of UEN and Phone Number

In [39]:
# Boolean masks for duplicates
uen_dup = New_Fresh_Leads_Operational["UEN"].duplicated(keep=False)

phone_dup = (
    New_Fresh_Leads_Operational["Phones"].notna() &
    New_Fresh_Leads_Operational["Phones"].duplicated(keep=False)
)

# YES/NO summary
print(
    "UEN dup:", "YES" if uen_dup.any() else "NO",
    "| Phone dup:", "YES" if phone_dup.any() else "NO"
)

# Show duplicate rows if exist
if uen_dup.any():
    print("\n🔁 Duplicate UEN rows:")
    display(New_Fresh_Leads_Operational[uen_dup])

if phone_dup.any():
    print("\n📱 Duplicate Phone rows:")
    display(New_Fresh_Leads_Operational[phone_dup])


UEN dup: NO | Phone dup: YES

📱 Duplicate Phone rows:


,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,operational_street,operational_unit,operational_postal_code,operational_address
5,202515699G,success,None,None,[+6562220308],https://www.pureblissspa.com.sg/,None,None,None,None,https://recordowl.com/company/pure-bliss-beaut...,177 TOA PAYOH CENTRAL,01-146,310177,177 TOA PAYOH CENTRAL 01-146 Singapore 310177
31,53369718B,success,None,[gergertbeauty@gmail.com],[+6585911557],https://gergertbeauty.com/,[https://www.facebook.com/gergert.singapore.be...,None,"[https://www.instagram.com/gergert.singapore/,...",None,https://recordowl.com/company/brow-lashes-gergert,6 EU TONG SEN STREET THE CENTRAL,08-10,059817,6 EU TONG SEN STREET THE CENTRAL 08-10 Singapo...
75,202022677K,success,None,None,[+6562821979],None,None,None,None,None,https://recordowl.com/company/la-bella-centre-...,203 HOUGANG STREET 21,01-75,530203,203 HOUGANG STREET 21 01-75 Singapore 530203
88,201612856C,success,None,None,[+6562220308],https://pureblissbcs.com/,"[https://www.facebook.com/purebliss308kovan, h...",None,None,None,https://recordowl.com/company/pure-bliss-spa-p...,17 SIMON ROAD,01-01,545902,17 SIMON ROAD 01-01 Singapore 545902
126,201707264M,success,None,None,[+6562220308],https://pureblissbcs.com/,None,None,None,None,https://recordowl.com/company/pure-bliss-spa-y...,411 RIVER VALLEY ROAD,01-01,248309,411 RIVER VALLEY ROAD 01-01 Singapore 248309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610,202450838R,success,None,None,[+6584841909],https://sunshinebeautyhub.com/,None,None,[https://www.instagram.com/sunshine_wellness_sg/],None,https://recordowl.com/company/sunshine-beauty-...,603 CLEMENTI WEST STREET 1,01-36,120603,603 CLEMENTI WEST STREET 1 01-36 Singapore 120603
2644,53460574E,success,None,None,[+6588958227],None,None,None,[https://www.instagram.com/illumeskinhouse/],None,https://recordowl.com/company/illumeskin,10 SINARAN DRIVE SQUARE 2,03-37,307506,10 SINARAN DRIVE SQUARE 2 03-37 Singapore 307506
2657,202324085W,success,None,None,[+6580436271],https://www.mycareersfuture.gov.sg/job/custome...,[https://www.facebook.com/DeNaturalBeauty/],None,None,None,https://recordowl.com/company/de-natural-beaut...,180 BENCOOLEN STREET THE BENCOOLEN,02-10,189646,180 BENCOOLEN STREET THE BENCOOLEN 02-10 Singa...
2702,202231810R,success,None,None,[+6580316028],None,[https://www.facebook.com/luxurybrowandbeauty],None,[https://www.instagram.com/luxurybrowandbeauty...,None,https://recordowl.com/company/luxury-brow-and-...,2 KALLANG AVENUE CT HUB,01-15,339407,2 KALLANG AVENUE CT HUB 01-15 Singapore 339407


### Drop duplicate phone numbers

In [40]:
# Convert list-like Phones into strings for comparison
New_Fresh_Leads_Operational["Phones_str"] = (
    New_Fresh_Leads_Operational["Phones"].astype(str)
)

# Create a NEW DataFrame with duplicate phone numbers removed
New_Fresh_Leads_Operational_unique_phones = (
    New_Fresh_Leads_Operational.drop_duplicates(
        subset="Phones_str", keep="first"
    )
    .drop(columns=["Phones_str"])  # clean up helper column
)

# Show size change
print("Original:", len(New_Fresh_Leads_Operational))
print("Unique Phones:", len(New_Fresh_Leads_Operational_unique_phones))


Original: 2768
Unique Phones: 1079


In [41]:
New_Fresh_Leads_Operational_x = New_Fresh_Leads_Operational.copy()

In [42]:
New_Fresh_Leads_Operational_x.to_parquet("./Staging/Silver/Silver_data_2768_spa.parquet", index=False, engine="fastparquet")